# 4.2 Modelado de un Vehículo de Transmisión Diferencial (Duckiebot)

## Tabla de Contenidos
1. Introducción a los Modelos Matemáticos
2. Representación del Estado
3. Cinemática (Directa e Inversa)
4. Modelo del Motor DC

---

## 1. Introducción a los Modelos Matemáticos

Un **modelo matemático** es una representación abstracta de un sistema real que captura las relaciones causales entre sus componentes, específicamente mapeando las variables de **Entrada (Inputs)** a las de **Salida (Outputs)**.

### 1.1 Objetivos del Modelado
No buscamos una descripción exacta de la realidad, sino cuantificar variables esenciales para:
* **Predicción:** Anticipar el comportamiento futuro del sistema (e.g., estimación de pose).
* **Control:** Manipular las entradas ($u_t$) para gobernar la dinámica y obtener la salida deseada.

### 1.2 Tipología de Modelos
1.  **Basados en Datos (Data-Driven / Black-box):**
    * Aprenden la correlación entrada-salida a partir de datasets (experiencia) sin conocimiento explícito de la física subyacente.
    * *Ejemplo:* Redes neuronales (como se ve en la primera imagen del diagrama).
2.  **Basados en Principios Físicos (Mecanicistas / White-box):**
    * Derivados de leyes fundamentales (ej. 2da Ley de Newton, Leyes de Kirchhoff). Son deterministas y explicables.
    * *Ejemplo:* El diagrama de bloques de control clásico.

---

### 1.3 Representación en Espacio de Estados
En este curso, nos centraremos en modelos físicos descritos mediante ecuaciones de estado generales:

$$
\dot{x}_t = f(x_t, u_t)
$$

$$
y_t = g(x_t, u_t) 
$$

Donde:
* $x_t$: **Vector de Estado** (variables internas mínimas para describir el sistema).
* $u_t$: **Vector de Control** (entradas, e.g., voltajes).
* $y_t$: **Vector de Salida** (mediciones o variables de interés).

<div align="center">
    <img src="imagenes/Pasted image 20260209205921.png" alt="Modelo basado en datos (Red Neuronal)" width="500"/>
    <p><em>Fig 1. Modelo basado en datos (Red Neuronal) vs. Modelo Físico</em></p>
    <br>
    <img src="imagenes/models.png" alt="Diagrama de bloques del sistema físico" width="700"/>
    <p><em>Fig 2. Esquema de control basado en modelo físico</em></p>
</div>

## 2. Plataforma: El Robot de Accionamiento Diferencial (Duckiebot)

El Duckiebot opera bajo una topología de **accionamiento diferencial**, donde el movimiento se logra mediante la diferencia de velocidades entre dos ruedas motrices independientes.

### 2.1 Configuración Física y Flujo de Señales
El sistema convierte energía eléctrica en movimiento espacial a través de la siguiente cadena de transformación:

1.  **Entrada (Control):** Voltajes $V_{l,t}, V_{r,t}$ aplicados a los motores DC.
2.  **Dinámica del Actuador:** Generación de Torque ($\tau$) y consecuente Velocidad Angular ($\dot{\phi}_{l,t}, \dot{\phi}_{r,t}$) en las ruedas.
3.  **Cinemática:** La interacción rueda-suelo transforma el giro en velocidad lineal y angular del chasis.
4.  **Salida (Estado):** Evolución del vector de Pose $\mathbf{q}_t = [x, y, \theta]^T$.

<div align="center">
    <img src="imagenes/Pasted image 20260209202936.png" alt="Esquema Differential Drive" width="600"/>
    <p><em>Fig 3. Diagrama de bloques: De Voltaje a Pose</em></p>
</div>

### 2.2 Preguntas Fundamentales

Esta sección define los dos problemas principales de la robótica móvil aplicados al Duckiebot:

**1. (Cinemática Directa)** Dados las velocidades de las ruedas $\dot{\phi}_1, \dot{\phi}_2, \dots, \dot{\phi}_t$, ¿cómo se moverá el robot en el espacio $(x, y, \theta)$?

**2. (Cinemática Inversa)** Dado un movimiento o trayectoria deseada $(q_1, \dots, q_t)$, ¿qué comandos (velocidades o voltajes) debemos enviar a las ruedas?

### 2.3 Notaciones y Suposiciones

Para modelar el movimiento matemáticamente, definimos los marcos de referencia y las restricciones físicas del sistema.

* **Marco Global (World Frame):** $\{x^w, y^w\}$ — Fijo en el entorno.
* **Marco del Cuerpo (Body/Robot Frame):** $\{x^r, y^r\}$ — Se mueve con el robot.

#### Suposición 1: Simetría
El robot es simétrico a lo largo del eje longitudinal ($x^r$).
* Las ruedas están equidistantes del centro (longitud del eje = $2L$).
* Las ruedas son idénticas (radio $R_l = R_r = R$).
* El centro de masa se encuentra en el eje $x^r$ a una distancia $c$ desde el punto $A$.

#### Suposición 2: Cuerpo Rígido
El robot se comporta como un cuerpo rígido ideal.
* La distancia entre dos puntos cualesquiera del robot no cambia con el tiempo.
* Matemáticamente: $\dot{c} = 0$, donde $(\dot{\star}) = \frac{d(\star)}{dt}$.

<div align="center">
    <img src="imagenes/Pasted image 20260209203048.png" alt="Diagrama del chasis del robot" width="500"/>
</div>

## 3. Cinemática y Geometría del Robot

Antes de formular las ecuaciones de movimiento, debemos definir los problemas a resolver y la geometría del sistema.

### 3.1 Problemas Fundamentales
Buscamos establecer las relaciones matemáticas para responder a dos necesidades de ingeniería:
1.  **Cinemática Directa (Predicción):** Dados los estados de los actuadores (velocidades de rueda $\dot{\phi}$), determinar la evolución del robot en el espacio $(\dot{x}, \dot{y}, \dot{\theta})$.
2.  **Cinemática Inversa (Control):** Dada una velocidad deseada del chasis, calcular las consignas de control para cada motor.

### 3.2 Marcos de Referencia y Pose
Para cuantificar el movimiento, definimos dos marcos ortonormales:
* **Marco Global ($\{W\}$):** Sistema inercial fijo en el entorno $(x_W, y_W)$.
* **Marco del Cuerpo ($\{R\}$):** Sistema móvil solidario al chasis $(x_R, y_R)$.

La **Pose** ($\mathbf{q}_t$) define el estado completo del vehículo en el plano:
$$\mathbf{q}_t = [x_t, y_t, \theta_t]^T \in SE(2)$$

### 3.3 Suposiciones Geométricas (Topología)
Basado en el diagrama del chasis, asumimos un **Cuerpo Rígido** con simetría longitudinal:
* **Punto $A$ (Centro de Eje):** Origen del marco $\{R\}$, equidistante a las ruedas.
* **Punto $C$ (Centro de Masa):** Ubicado en el eje longitudinal $x_R$, desplazado una distancia $c$ respecto a $A$.
* **Parámetros:**
    * $R$: Radio de las ruedas (asumido idéntico).
    * $L$: Semiancho de vía (distancia lateral de $A$ a la rueda).

<div align="center">
    <img src="imagenes/Pasted image 20260209203048.png" alt="Geometría del Duckiebot y Marcos de Referencia" width="600"/>
</div>

### 4. Ecuaciones Cinemáticas Generales

Una vez que entendemos cómo se mueven las ruedas, debemos traducir ese movimiento al **Marco Global**. Esto se logra mediante la matriz de rotación $R(\theta)$.

#### Transformación de Velocidades
La velocidad del punto de referencia $A$ en el mundo ($\mathbf{v}_A^w$) se relaciona con la velocidad en el marco del robot ($\mathbf{v}_A^r$) rotando el vector por el ángulo actual $\theta$:

$$
\mathbf{v}_A^w = \begin{bmatrix} \dot{x} \\ \dot{y} \end{bmatrix} = R(\theta) \mathbf{v}_A^r = \underbrace{\begin{bmatrix} \cos\theta & -\sin\theta \\ \sin\theta & \cos\theta \end{bmatrix}}_{R(\theta)} \begin{bmatrix} v_u \\ v_v - c\dot{\theta} \end{bmatrix}
$$

De esto obtenemos el **Sistema de Ecuaciones Cinemáticas**:

$$
\begin{cases} 
\dot{x} = v_u \cos\theta - (v_v - c\dot{\theta}) \sin\theta \\ 
\dot{y} = v_u \sin\theta + (v_v - c\dot{\theta}) \cos\theta \\ 
\dot{\theta} = \omega 
\end{cases}
$$

Donde $\omega$ es la velocidad angular del chasis alrededor del eje $z$.

### 5. Restricciones Cinemáticas

Para que el modelo sea válido en Duckietown (y en la mayoría de robots de ruedas), asumimos condiciones de "rodadura ideal".

#### 1. Sin Deslizamiento Lateral (No Skidding)
El robot no puede deslizarse de lado (como un coche derrapando en hielo). Esto implica que la velocidad lateral en el punto de contacto es cero.
$$
\mathbf{v}_A^r = \begin{bmatrix} v_u \\ 0 \end{bmatrix} \quad \Rightarrow \quad v_v = c\dot{\theta}
$$
*(Si el centro de masa está en el eje de las ruedas, $c=0$, entonces $v_v=0$).*

#### 2. Sin Patinaje de Rueda (No Slipping)
La rueda no gira en falso. En cada revolución completa ($\Delta\phi = 2\pi$), la rueda avanza exactamente su perímetro ($\Delta x = 2\pi R$).

La relación fundamental entre la velocidad lineal de la rueda ($v_{l/r}$) y su velocidad angular ($\dot{\phi}_{l/r}$) es:

$$
v_{l/r} = \frac{\Delta x}{\Delta t} = \dot{\phi}_{l/r} R
$$

<div align="center">
    <img src="imagenes/Pasted image 20260209203302.png" alt="Relación velocidad angular y lineal" width="600"/>
</div>

### 6. Rotación de Cuerpo Rígido (Rigid Body Rotation)

Dado que las ruedas izquierda y derecha pueden girar a velocidades diferentes, el robot rota alrededor de un punto pivote llamado **ICC (Centro Instantáneo de Curvatura)** o ICR (Centro Instantáneo de Rotación).

* Si $v_l = v_r$: El robot va recto (ICC en el infinito).
* Si $v_l = -v_r$: El robot gira sobre su propio eje (ICC en el centro del eje).
* Si $v_l \neq v_r$: El robot describe una curva alrededor del ICC.

La velocidad angular del chasis ($\omega = \dot{\theta}$) depende de la diferencia de velocidades de las ruedas y el ancho del eje ($2L$):

$$
\omega = \frac{v_r - v_l}{2L}
$$

<div align="center">
    <img src="imagenes/Pasted image 20260209203655.png" alt="Concepto de ICC" width="400"/>
    <br>
    <img src="imagenes/Pasted image 20260209203752.png" alt="Geometría del ICC y velocidades" width="400"/>
</div>

### 7. Integración: Del Voltaje a la Pose (Duckiebot Model)

Finalmente, combinamos el modelo de los actuadores (Motores DC) con la geometría del robot para obtener el modelo completo de **Cinemática Directa**.

El objetivo es encontrar la evolución del estado $\dot{\mathbf{q}}_t$ basándonos en los voltajes de entrada $V_{l,t}$ y $V_{r,t}$.

<div align="center">
    <img src="imagenes/Pasted image 20260209211407.png" alt="Modelo completo del Duckiebot" width="800"/>
</div>

#### Ecuación Matricial Completa

La evolución del sistema se describe multiplicando las matrices de rotación y geometría:

$$
\dot{\mathbf{q}}_t = \underbrace{\frac{R}{2} \begin{bmatrix} 
\cos\theta & 0 \\ 
\sin\theta & 0 \\ 
0 & 1 
\end{bmatrix}}_{\text{Transformación a Marco Global}} 
\underbrace{\begin{bmatrix} 
1 & 1 \\ 
\frac{1}{L} & -\frac{1}{L} 
\end{bmatrix}}_{\text{Geometría del Chasis}} 
\begin{bmatrix} 
\dot{\phi}_r \\ 
\dot{\phi}_l 
\end{bmatrix}
$$

Donde las velocidades de las ruedas $\dot{\phi}$ se obtienen del modelo del motor que vimos anteriormente:

$$
\dot{\phi}_t = \left( R \frac{b}{K_i} + K_b \right)^{-1} V_t
$$

**Interpretación de las matrices:**
1.  **Matriz derecha:** Convierte las velocidades individuales de las ruedas en velocidad lineal ($v$) y angular ($\omega$) del robot.
2.  **Matriz izquierda:** Proyecta esas velocidades ($v, \omega$) en las coordenadas globales $(x, y, \theta)$ usando la orientación actual.

In [3]:
import numpy as np

def forward_kinematics(phi_dot_left, phi_dot_right, theta, R, L):
    """
    Calcula el cambio en la pose (x_dot, y_dot, theta_dot)
    basado en las velocidades de las ruedas y la orientación actual.
    
    Args:
        phi_dot_left (float): Velocidad angular rueda izquierda [rad/s]
        phi_dot_right (float): Velocidad angular rueda derecha [rad/s]
        theta (float): Orientación actual del robot [rad]
        R (float): Radio de la rueda [m]
        L (float): Mitad de la distancia entre ruedas (baseline/2) [m]
        
    Returns:
        np.array: Vector velocidad del estado [x_dot, y_dot, theta_dot]
    """
    # Vector de velocidades de ruedas
    wheel_speeds = np.array([
        [phi_dot_right],
        [phi_dot_left]
    ])
    
    # Matriz de Geometría (Ruedas -> v, omega)
    # Nota: Multiplicamos por R/2 aquí según la fórmula de la imagen
    geometry_matrix = np.array([
        [1, 1],
        [1/L, -1/L]
    ])
    
    # Matriz de Rotación (v, omega -> x_dot, y_dot, theta_dot)
    rotation_matrix = np.array([
        [np.cos(theta), 0],
        [np.sin(theta), 0],
        [0, 1]
    ])
    
    # Cálculo encadenado: q_dot = (R/2) * Rot * Geom * Wheels
    # Paso 1: Velocidades en el marco del robot (v, omega)
    # Factorizamos R/2 que multiplica a todo
    robot_frame_vels = (R / 2) * np.dot(geometry_matrix, wheel_speeds)
    
    # Paso 2: Velocidades en el marco global (x_dot, y_dot, theta_dot)
    # Nota: La matriz de rotación en la imagen es de 3x2 para coincidir con el vector v, omega
    global_vels = np.dot(rotation_matrix, robot_frame_vels)
    
    return global_vels.flatten() # Retorna vector plano (x_dot, y_dot, theta_dot)

# --- PRUEBA ---
R_duckie = 0.0318 # Radio típico Duckiebot (m)
L_duckie = 0.05   # Mitad del eje (m)
theta_actual = np.pi / 4 # 45 grados

# Caso: Avanzando y girando levemente a la izquierda
q_dot = forward_kinematics(phi_dot_left=5.0, phi_dot_right=6.0, 
                          theta=theta_actual, R=R_duckie, L=L_duckie)

print(f"Cambio en X (m/s): {q_dot[0]:.4f}")
print(f"Cambio en Y (m/s): {q_dot[1]:.4f}")
print(f"Cambio en Theta (rad/s): {q_dot[2]:.4f}")

Cambio en X (m/s): 0.1237
Cambio en Y (m/s): 0.1237
Cambio en Theta (rad/s): 0.3180


### 3.1 Dinámica Detallada de los Motores (Subsistemas)

Para un control de bajo nivel más preciso, no solo miramos la velocidad final, sino cómo llegamos a ella. Esto implica considerar el comportamiento eléctrico (inductancia) y mecánico (inercia).

#### Subsistema Eléctrico
Relaciona el voltaje con la corriente, considerando la resistencia y la inductancia de la bobina:

$$
V_t = R i_t + L_e \frac{di_t}{dt} + K_b \dot{\phi}_t
$$

* $K_b \dot{\phi}_t$: **Fuerza contraelectromotriz (Back e.m.f.)**. Voltaje generado por el propio giro del motor que se opone a la entrada.
* $\tau_t = K_i i_t$: El torque generado es proporcional a la corriente.

#### Subsistema Mecánico
Relaciona el torque con la aceleración angular, considerando la inercia del rotor y la fricción:

$$
J \ddot{\phi}_t = \tau_t - b \dot{\phi}_t
$$

* $J$: Inercia del rotor.
* $b \dot{\phi}_t$: **Fricción viscosa**.

## Resumen del Modelo Duckiebot

Para consolidar lo visto, definimos el sistema completo bajo las siguientes restricciones y flujos.

### Suposiciones de Diseño
Para que el modelo cinemático y dinámico que usaremos sea válido, asumimos:

1.  **Simetría:** El robot es simétrico a lo largo del eje longitudinal ($x^r$).
2.  **Cuerpo Rígido:** El chasis no se deforma.
3.  **Rodadura Pura:** Las ruedas **no se deslizan** (lateralmente) **ni patinan** (longitudinalmente).

---

### Diagrama de Bloques del Sistema

El siguiente diagrama resume el flujo completo de la señal, desde el código (voltaje) hasta el movimiento en el mundo real (pose):

1.  **Entradas ($V_{l,t}, V_{r,t}$):** Voltajes aplicados a los motores.
2.  **Actuadores (Motores DC):** Convierten voltaje en velocidad angular ($\dot{\phi}$).
3.  **Cinemática Directa:** Convierte el giro de las ruedas en velocidad del chasis ($v, \omega$) y actualiza la pose.
4.  **Salida ($\mathbf{q}_t$):** Vector de estado $(x_t, y_t, \theta_t)^T$.

<div align="center">
    <img src="imagenes/Pasted image 20260209211407.png" alt="Diagrama de bloques Duckiebot" width="800"/>
</div>

#### Ecuaciones del Diagrama
Como se ve en la figura, el sistema se rige por:

**1. Dinámica del Motor (Simplificada):**
$$\dot{\phi}_t = \left( R \frac{b}{K_i} + K_b \right)^{-1} V_t$$

**2. Cinemática Diferencial:**
$$\dot{\mathbf{q}}_t = \frac{R}{2} \begin{bmatrix} \cos\theta & 0 \\ \sin\theta & 0 \\ 0 & 1 \end{bmatrix} \begin{bmatrix} 1 & 1 \\ \frac{1}{L} & -\frac{1}{L} \end{bmatrix} \begin{bmatrix} \dot{\phi}_r \\ \dot{\phi}_l \end{bmatrix}$$